In [18]:
#based on https://github.com/freqtrade/freqtrade-strategies/blob/master/user_data/strategies/berlinguyinca/Scalp.py
from typing import Dict, List
from functools import reduce
from pandas import DataFrame
# --------------------------------
import talib.abstract as tab
from typing import Dict, List
from functools import reduce
from pandas import DataFrame, DatetimeIndex, merge
# --------------------------------
import talib as ta
import numpy  # noqa

#to insert config folder
import sys
sys.path.insert(0, '../')

In [20]:
class Scalp(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    def populateindicators(dataframe) -> DataFrame:
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=5, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=5, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=5, price='low')
        stoch_fast = tab.STOCHF(dataframe, 10.0, 3.0, 0.0, 3.0, 0.0)
        dataframe['fastd'] = stoch_fast['fastd']
        dataframe['fastk'] = stoch_fast['fastk']
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing
        bollinger =ta.BBANDS(data.close,timeperiod=10)
        dataframe['bb_lowerband'] = bollinger[2]
        dataframe['bb_upperband'] = bollinger[0]
        dataframe['bb_middleband'] = bollinger[1]

        return dataframe
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        self.indics=populateindicators(self.data)
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy


    def next(self):
        if not self.position:
            if self.indics['open'] < self.indics['ema_low'] and \
            self.indics['adx'] > 30 and \
            self.indics['fastk'] < 30 and \
            self.indics['fastd'] < 30:
                self.buy(size=100)
                
                    
        else:
            if self.indics['open'] >= self.indics['ema_high'] and \
            self.indics['fastk'] > 70 and \
            self.indics['fastd']> 70:
                self.sell(size=100)




In [21]:
'''   def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
                (dataframe['open'] < dataframe['ema_low']) &
                (dataframe['adx'] > 30) &
                (
                    (dataframe['fastk'] < 30) &
                    (dataframe['fastd'] < 30) &
                    (qtpylib.crossed_above(dataframe['fastk'], dataframe['fastd']))
                )
            ),
            'buy'] = 1
        return dataframe
    def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
                (dataframe['open'] >= dataframe['ema_high'])
            ) |
            (
                (qtpylib.crossed_above(dataframe['fastk'], 70)) |
                (qtpylib.crossed_above(dataframe['fastd'], 70))
            ),
            'sell'] = 1
        return dataframe
        
'''

"   def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:\n        dataframe.loc[\n            (\n                (dataframe['open'] < dataframe['ema_low']) &\n                (dataframe['adx'] > 30) &\n                (\n                    (dataframe['fastk'] < 30) &\n                    (dataframe['fastd'] < 30) &\n                    (qtpylib.crossed_above(dataframe['fastk'], dataframe['fastd']))\n                )\n            ),\n            'buy'] = 1\n        return dataframe\n    def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:\n        dataframe.loc[\n            (\n                (dataframe['open'] >= dataframe['ema_high'])\n            ) |\n            (\n                (qtpylib.crossed_above(dataframe['fastk'], 70)) |\n                (qtpylib.crossed_above(dataframe['fastd'], 70))\n            ),\n            'sell'] = 1\n        return dataframe\n        \n"